In [2]:
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup
import re
#from multiprocessing import Pool, Process
import os


In [3]:
def get_all_page(urls, pattern, url_base):

    if pattern in urls:
        return(urls)
    
    html = requests.get(pattern)
    soup = BeautifulSoup(html.content, "html.parser")
    last_page = soup.find("a", {"title" : "Dernière page"})["page"]

    iter = soup.find_all("div", {"class" : re.compile("m3-pagination print_hidden|print_hidden pagination")})
    for item in iter:
        lst = re.findall("url.*?;", item.find_next_sibling().text)
        nw_url = ""
        if len(lst) > 0:
            for item in lst:
                if re.search("querystring", item):
                    continue
                else:
                    item = re.sub("url =|url \+=|url", "", item)
                    item = item.replace(";", "").replace("\"", "").strip()
                    item = item.split(" + ")
                for part in item:
                    nw_url += part
            match = re.compile("\$\(this\)\.attr\(page\)")
            for i in range(1, int(last_page)+1):
                url = urljoin(url_base, re.sub(match,str(i), nw_url))
                if url not in urls:
                    urls.append(url)
                    
        else:
            for i in range(1, int(last_page)+1):
                urls.append(urljoin(url_base, re.sub("page=.", f"page={i}", pattern)))
    
    return(urls)

In [4]:
def getAllUrl(url, url_base, urls, EXPORT_FILE, DATA_FOLDER, page):

    soup = BeautifulSoup(page.content, "html.parser")
    for anchor in soup.find_all("a"):
        if "href" in anchor.attrs:
            if anchor["href"] == "JavaScript:void(0);" and anchor["class"] == "lien-page":
                try:
                    urls.extend(get_all_page(urls), url, url_base)
                except:
                    continue
            absolute_url = urljoin(url_base, anchor["href"])
            if "download" not in absolute_url and absolute_url not in urls and anchor["href"] != "JavaScript:void(0);":
                if re.search(url_base, absolute_url):
                    urls.append(absolute_url)
    # Export des pages html ?
                    with open(DATA_FOLDER+EXPORT_FILE, "a", encoding="utf-8") as f:
                        f.write(absolute_url + '\n')

    return(urls)

In [5]:
def read_urls(EXPORT_FILE):
    with open(EXPORT_FILE, "r") as f:
        urls = f.read().split("\n") 
    print(len(urls))
    return(urls)

In [21]:

EXPORT_FILE = "M3urls_mac.txt"
HTML_FOLDER = "urls/"
DATA_FOLDER = "data/"

if __name__ == "__main__":
    
    url_base = 'https://www.millenaire3.com'
    errors = []

    if EXPORT_FILE in os.listdir(DATA_FOLDER):
        urls = read_urls(DATA_FOLDER+EXPORT_FILE)
    else:
        urls = get_all_page(urls = [], 
                    pattern="https://www.millenaire3.com/m3_search?page=1&sort_by=0&SearchText=&modeRecherche=1&titre=&auteurId=&auteurText=&100%25=&100%25_new_value=true&date_debut=01%2F01%2F1900&date_debut_mobile=&date_fin=31%2F12%2F2025&date_fin_mobile=&identifiers=&rubriques=",
                    url_base = url_base)
        with open(DATA_FOLDER+EXPORT_FILE, "w", encoding="utf-8") as f:
            for url in urls:
                f.write(url+'\n')
        
    files = os.listdir(HTML_FOLDER)

    i = 15850 #len(files) ? # On peut changer la valeur de départ
    while i <= len(urls)-1:
        if f"{str(i)}.html" in files:
            pass
        elif re.search("/partage_email/|/generated_pdf/|", urls[i]):
            errors.append(urls[i])
            print(f"\r{i} / {len(urls)}, errors", end = "")
            pass
        else:
            print(f"\r{i} / {len(urls)}: {urls[i-1]}", end = "")

            page = requests.get(urls[i])
            if page.status_code == 200:
                getAllUrl(urls[i], url_base, urls, EXPORT_FILE, DATA_FOLDER, page)
                with open(f"{HTML_FOLDER}{str(i)}.html", "w", encoding="utf-8") as f:
                    f.write(page.text)
            else:
                print(" ", page.status_code)
                errors.append(urls[i])

        i += 1

15879
15878 / 15879, errors